# Topic: Shortest paths algorithms
## Introduction
* We can use structures from previous assignment
* |V| = n
* |V| = e ~ n^2
## Assignment objectives
* Implement Floyd-Warschall algorithm - can be implemeneted using matrix notation
* Implement Bellman-Ford algorithm
* Run experiments
* Run additional experiments
* Use histogram, box plot or violin plot

In [9]:
using BenchmarkTools
using Statistics
using Random
using StatsPlots

# Graph generating function
function generate_graph_fw(n::Int, edge_prob::Float64 = 0.8)
    A = fill(Inf, n, n)
    for i in 1:n
        A[i,i] = 0.0
        for j in 1:n
            if i != j && rand() < edge_prob
                # Allow negative weights and avoid negative self-loops
                A[i,j] = rand(-5.0:1.0:10.0) 
            end
        end
    end

    return A
end


# Main Algorithm Floyd Warshall
function floydwarshall(A::Matrix{Float64})
    n = size(A, 1)
    dist = copy(A)

    for k in 1:n
        for i in 1:n
            for j in 1:n
                if dist[i, j] > dist[i, k] + dist[k, j]
                    dist[i, j] = dist[i, k] + dist[k, j]
                end
            end
        end
    end

    return dist
end


# Time estimator for later experiments
function estimate_instances_fw(A::Matrix{Float64}, target_time::Float64 = 420.0)
    sample_runs = 3
    times = [@elapsed floydwarshall(copy(A)) for _ in 1:sample_runs]
    avg_time = mean(times)
    est_instances = max(1, Int(round(target_time / avg_time)))

    return est_instances
end


# Time measuring function
function measure_fw_time(A::Matrix{Float64}, num_instances::Int)
    times = [@elapsed floydwarshall(copy(A)) for _ in 1:num_instances]
    avg_time = mean(times)
    median_time = median(times)
    total_time = sum(times)
    
    return avg_time, median_time, total_time, times
end

# Function for runnig whole experiment using additional functions
function run_experiment(edge_prob::Float64 = 0.8, target_time::Float64 = 420.0, num_runs_per_size::Int = 50)
    sizes = [10, 50, 100, 200, 400, 600, 800, 1000]
    all_times = Vector{Vector{Float64}}()
    
    for n in sizes
        times = []
        for _ in 1:num_runs_per_size
            graph = generate_graph_fw(n)
            t = @elapsed floydwarshall(graph)
            push!(times, t)
        end
        push!(all_times, times)
    end
    
    violin(
        sizes, 
        all_times, 
        xlabel="Graph Size (n)", 
        ylabel="Time (s)",
        title="Floyd-Warshall Algorithm Performance",
        show_median=true
        )
    

    return sizes, all_times
end

run_experiment (generic function with 4 methods)

In [10]:
# Run Floyd Warshall Experiment
run_experiment(0.7)

# Violin plot results
# violin(sizes, all_times,
#        xlabel = "Graph Size (n)",
#        ylabel = "Computation Time (s)",
#        title = "Floyd-Warshall Timing Distribution",
#        show_median = true,
#        legend = false,
#        fillalpha = 0.7)

([10, 50, 100, 200, 400, 600, 800, 1000], [[5.3e-6, 4.2e-6, 4.9e-6, 4.7e-6, 5.0e-6, 4.1e-6, 4.7e-6, 4.2e-6, 4.7e-6, 4.6e-6  …  4.4e-6, 4.0e-6, 6.2e-6, 6.4e-6, 5.8e-6, 5.6e-6, 8.5e-6, 6.2e-6, 3.8e-6, 4.3e-6], [0.0004186, 0.000395, 0.0004135, 0.000407, 0.000408, 0.0004143, 0.0004096, 0.0004057, 0.0004049, 0.0004086  …  0.0004199, 0.0004209, 0.0004182, 0.0004237, 0.0004158, 0.0004156, 0.0004209, 0.0004176, 0.0004151, 0.0004075], [0.0034027, 0.0038831, 0.0037927, 0.0033561, 0.0034092, 0.0034466, 0.0053587, 0.0040488, 0.0035705, 0.0036818  …  0.0035366, 0.0037109, 0.0035878, 0.0034932, 0.0035184, 0.0035753, 0.0037905, 0.0037501, 0.0046179, 0.0041182], [0.0358338, 0.0284632, 0.0287988, 0.0295709, 0.0311099, 0.0288707, 0.0290329, 0.0290548, 0.0356924, 0.030419  …  0.0346282, 0.0382286, 0.036207, 0.0422181, 0.0431819, 0.0435756, 0.0386975, 0.0413564, 0.046765, 0.0434229], [0.2963917, 0.2592789, 0.2631504, 0.3806597, 0.2998062, 0.3687147, 0.2707194, 0.2626467, 0.2494462, 0.240477  …  0.238591, 